In [1]:
import pandas as pd
import numpy as np
import yfinance as yf


In [2]:
dataF = yf.download("SBIN.NS",interval='1h',period='2y')


[*********************100%%**********************]  1 of 1 completed


In [3]:
dataF

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-08-10 09:15:00+05:30,521.549988,523.400024,516.400024,518.200012,518.200012,0
2022-08-10 10:15:00+05:30,518.200012,518.849976,515.500000,515.750000,515.750000,1784831
2022-08-10 11:15:00+05:30,515.750000,517.250000,515.500000,516.400024,516.400024,1885971
2022-08-10 12:15:00+05:30,516.599976,517.400024,516.000000,516.150024,516.150024,1714948
2022-08-10 13:15:00+05:30,516.200012,516.450012,513.500000,514.750000,514.750000,2204517
...,...,...,...,...,...,...
2024-08-08 11:15:00+05:30,811.200012,816.750000,811.200012,816.750000,816.750000,2538224
2024-08-08 12:15:00+05:30,816.900024,817.799988,810.349976,811.299988,811.299988,1781582
2024-08-08 13:15:00+05:30,811.299988,811.900024,807.049988,809.000000,809.000000,1503106


In [4]:
data = dataF.copy().dropna()
data['20EMA'] = data['Close'].ewm(span=20, adjust=False).mean()
data['50EMA'] = data['Close'].ewm(span=50, adjust=False).mean()
data['200EMA'] = data['Close'].ewm(span=200, adjust=False).mean()

# Calculate Slopes for each EMA
data['Slope_20EMA'] = data['20EMA'].diff()
data['Slope_50EMA'] = data['50EMA'].diff()
data['Slope_200EMA'] = data['200EMA'].diff()

data = data.dropna()

In [5]:

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate MACD
def calculate_macd(data, slow=26, fast=12, signal=9):
    exp1 = data.ewm(span=fast, adjust=False).mean()
    exp2 = data.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

# Calculating RSI and MACD for the Close price
data['RSI'] = calculate_rsi(data['Close'])
data['MACD'], data['Signal_Line'] = calculate_macd(data['Close'])

# Display the data with new indicators
data

,Open,High,Low,Close,Adj Close,Volume,20EMA,50EMA,200EMA,Slope_20EMA,Slope_50EMA,Slope_200EMA,RSI,MACD,Signal_Line
Datetime,,,,,,,,,,,,,,,
2022-08-10 10:15:00+05:30,518.200012,518.849976,515.500000,515.750000,515.750000,1784831,517.966678,518.103933,518.175634,-0.233334,-0.096079,-0.024378,NaN,0.000000,0.000000
2022-08-10 11:15:00+05:30,515.750000,517.250000,515.500000,516.400024,516.400024,1885971,517.817473,518.037113,518.157966,-0.149205,-0.066820,-0.017668,NaN,0.051854,0.010371
2022-08-10 12:15:00+05:30,516.599976,517.400024,516.000000,516.150024,516.150024,1714948,517.658668,517.963110,518.137987,-0.158805,-0.074003,-0.019980,NaN,0.071946,0.022686
2022-08-10 13:15:00+05:30,516.200012,516.450012,513.500000,514.750000,514.750000,2204517,517.381652,517.837106,518.104275,-0.277016,-0.126004,-0.033711,NaN,-0.024815,0.013186
2022-08-10 14:15:00+05:30,514.799988,515.900024,513.700012,515.099976,515.099976,3508643,517.164350,517.729767,518.074382,-0.217303,-0.107338,-0.029894,NaN,-0.072424,-0.003936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-08 11:15:00+05:30,811.200012,816.750000,811.200012,816.750000,816.750000,2538224,813.272784,830.296736,848.031158,0.366023,-0.552928,-0.314383,51.612926,-7.353844,-10.449827
2024-08-08 12:15:00+05:30,816.900024,817.799988,810.349976,811.299988,811.299988,1781582,813.084899,829.551765,847.665674,-0.187885,-0.744971,-0.365484,49.025082,-6.681760,-9.696214
2024-08-08 13:15:00+05:30,811.299988,811.900024,807.049988,809.000000,809.000000,1503106,812.695861,828.745814,847.280940,-0.389038,-0.805952,-0.384733,56.110514,-6.262529,-9.009477


In [6]:
# Initialize the 'Signal' column to zero
data['Signal'] = 0

# Define buy signals (RSI < 30 and MACD crosses above signal line)
buy_signals = ((data['RSI'] < 30) & (data['MACD'] > data['Signal_Line']) & (data['MACD'].shift(1) < data['Signal_Line'].shift(1)))

# Define sell signals (RSI > 70 and MACD crosses below signal line)
sell_signals = ((data['RSI'] > 70) & (data['MACD'] < data['Signal_Line']) & (data['MACD'].shift(1) > data['Signal_Line'].shift(1)))

# Assign signals to the 'Signal' column
data.loc[buy_signals, 'Signal'] = 1
data.loc[sell_signals, 'Signal'] = -1

# Display data with the new 'Signal' column


In [7]:
data[data['Signal'] !=0]

,Open,High,Low,Close,Adj Close,Volume,20EMA,50EMA,200EMA,Slope_20EMA,Slope_50EMA,Slope_200EMA,RSI,MACD,Signal_Line,Signal
Datetime,,,,,,,,,,,,,,,,
2022-09-05 13:15:00+05:30,539.150024,540.200012,538.049988,539.849976,539.849976,874347,535.461178,530.152984,523.660982,0.461979,0.395796,0.162703,81.422671,3.459543,3.474137,-1
2022-09-15 15:15:00+05:30,571.650024,572.250000,570.099976,571.500000,571.500000,676140,567.332092,557.323408,537.201425,0.438727,0.578636,0.344709,70.977810,5.442148,5.700644,-1
2022-09-20 15:15:00+05:30,573.299988,575.250000,573.200012,574.049988,574.049988,576543,571.882501,565.216112,543.502023,0.228156,0.360566,0.307015,70.661956,2.794022,2.803045,-1
2022-10-19 12:15:00+05:30,554.150024,554.500000,551.549988,553.299988,553.299988,1052046,549.849570,541.407780,539.155464,0.363202,0.485396,0.142156,72.327072,6.640218,6.970243,-1
2022-10-27 14:15:00+05:30,581.500000,581.599976,578.450012,579.500000,579.500000,1996915,573.886018,562.058677,546.741058,0.590945,0.711891,0.329236,74.757298,6.712772,6.942288,-1
2022-11-22 14:15:00+05:30,598.799988,600.000000,598.250000,599.000000,599.000000,1995885,598.746660,598.677668,581.170846,0.026667,0.013156,0.179187,29.556333,-0.385507,-0.389251,1
2023-01-02 11:15:00+05:30,614.000000,614.950012,613.349976,614.599976,614.599976,575069,610.547136,605.132119,602.103097,0.426615,0.386443,0.125597,71.926614,4.321779,4.476797,-1
2023-01-11 14:15:00+05:30,595.849976,598.500000,595.349976,596.599976,596.599976,1423937,598.989841,602.177583,602.848530,-0.251565,-0.227657,-0.062800,27.938330,-2.367723,-2.377162,1
2023-01-30 15:15:00+05:30,540.849976,541.900024,537.400024,541.299988,541.299988,2485783,549.989508,568.784065,590.246173,-0.914686,-1.121799,-0.491921,29.598150,-13.105954,-13.533550,1


In [8]:
# Calculate differences between slopes to determine convergence/divergence
data['Slope_Diff_20_50'] = data['Slope_20EMA'] - data['Slope_50EMA']
data['Slope_Diff_20_200'] = data['Slope_20EMA'] - data['Slope_200EMA']
data['Slope_Diff_50_200'] = data['Slope_50EMA'] - data['Slope_200EMA']

# Define additional buy signals based on slope convergence/divergence
# Convergence condition: Slopes of 20EMA and 50EMA are increasing while 200EMA slope is more negative or stable
slope_convergence_buy = (
    (data['Slope_20EMA'] > data['Slope_50EMA']) & 
    (data['Slope_50EMA'] > data['Slope_200EMA']) & 
    (data['Slope_20EMA'] > 0) & (data['Slope_50EMA'] > 0)
)

# Divergence condition: 20EMA slope is positive and diverging from more negative or less positive 50EMA and 200EMA slopes
slope_divergence_buy = (
    (data['Slope_20EMA'] > 0) & 
    (data['Slope_50EMA'] < data['Slope_20EMA']) & 
    (data['Slope_200EMA'] < data['Slope_20EMA'])
)

# Combine RSI, MACD, and slope conditions for a robust buy signal
data['Combined_Buy_Signal'] = (
    (buy_signals | slope_convergence_buy | slope_divergence_buy) & (data['RSI'] < 30) & (data['MACD'] > data['Signal_Line'])
)

# Mark the combined buy signal in the 'Signal' column
data.loc[data['Combined_Buy_Signal'], 'Signal'] = 1

# Display data with the new combined buy signal
data

,Open,High,Low,Close,Adj Close,Volume,20EMA,50EMA,200EMA,Slope_20EMA,Slope_50EMA,Slope_200EMA,RSI,MACD,Signal_Line,Signal,Slope_Diff_20_50,Slope_Diff_20_200,Slope_Diff_50_200,Combined_Buy_Signal
Datetime,,,,,,,,,,,,,,,,,,,,
2022-08-10 10:15:00+05:30,518.200012,518.849976,515.500000,515.750000,515.750000,1784831,517.966678,518.103933,518.175634,-0.233334,-0.096079,-0.024378,NaN,0.000000,0.000000,0,-0.137256,-0.208956,-0.071701,False
2022-08-10 11:15:00+05:30,515.750000,517.250000,515.500000,516.400024,516.400024,1885971,517.817473,518.037113,518.157966,-0.149205,-0.066820,-0.017668,NaN,0.051854,0.010371,0,-0.082385,-0.131537,-0.049152,False
2022-08-10 12:15:00+05:30,516.599976,517.400024,516.000000,516.150024,516.150024,1714948,517.658668,517.963110,518.137987,-0.158805,-0.074003,-0.019980,NaN,0.071946,0.022686,0,-0.084801,-0.138825,-0.054024,False
2022-08-10 13:15:00+05:30,516.200012,516.450012,513.500000,514.750000,514.750000,2204517,517.381652,517.837106,518.104275,-0.277016,-0.126004,-0.033711,NaN,-0.024815,0.013186,0,-0.151012,-0.243305,-0.092293,False
2022-08-10 14:15:00+05:30,514.799988,515.900024,513.700012,515.099976,515.099976,3508643,517.164350,517.729767,518.074382,-0.217303,-0.107338,-0.029894,NaN,-0.072424,-0.003936,0,-0.109964,-0.187409,-0.077445,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-08 11:15:00+05:30,811.200012,816.750000,811.200012,816.750000,816.750000,2538224,813.272784,830.296736,848.031158,0.366023,-0.552928,-0.314383,51.612926,-7.353844,-10.449827,0,0.918951,0.680406,-0.238544,False
2024-08-08 12:15:00+05:30,816.900024,817.799988,810.349976,811.299988,811.299988,1781582,813.084899,829.551765,847.665674,-0.187885,-0.744971,-0.365484,49.025082,-6.681760,-9.696214,0,0.557085,0.177599,-0.379486,False
2024-08-08 13:15:00+05:30,811.299988,811.900024,807.049988,809.000000,809.000000,1503106,812.695861,828.745814,847.280940,-0.389038,-0.805952,-0.384733,56.110514,-6.262529,-9.009477,0,0.416914,-0.004305,-0.421219,False


In [9]:
buy_signal_data = data[data['Combined_Buy_Signal']]

In [15]:
# Update the days to hold from 5 to 30 days
days_to_hold = 30  # Days to hold the trade
stop_loss_threshold = 0.1 # 5%
target_profit_threshold = 0.25  # 10%
days_to_hold = 45  # Days to hold the trade

# Initialize lists to store trade outcomes
trade_results = []
profit_losses = []

# Reinitialize lists to store trade outcomes for the new backtest
trade_results_extended = []
profit_losses_extended = []

  


In [16]:
def backtest(data, days_to_hold, stop_loss_threshold, target_profit_threshold):
    buy_signal_data = data[data['Combined_Buy_Signal']]
    trade_results_extended = []
    profit_losses_extended = []

    for index, row in buy_signal_data.iterrows():
        buy_price = row['Close']
        entry_date = index
        exit_date = entry_date + pd.DateOffset(days=days_to_hold)

        trade_window_extended = data[(data.index > entry_date) & (data.index <= exit_date)]

        if trade_window_extended.empty:
            continue

        sold_price_extended = None
        hit_stop_loss_extended = False
        hit_target_profit_extended = False

        for i, day in trade_window_extended.iterrows():
            low_price = day['Low']
            high_price = day['High']
            
            if low_price <= buy_price * (1 - stop_loss_threshold):
                sold_price_extended = buy_price * (1 - stop_loss_threshold)
                hit_stop_loss_extended = True
                break
            
            if high_price >= buy_price * (1 + target_profit_threshold):
                sold_price_extended = buy_price * (1 + target_profit_threshold)
                hit_target_profit_extended = True
                break
        
        if not sold_price_extended:
            sold_price_extended = trade_window_extended.iloc[-1]['Close']
        
        profit_loss_extended = (sold_price_extended - buy_price) / buy_price
        profit_losses_extended.append(profit_loss_extended)

        trade_results_extended.append({
            'Entry Date': entry_date,
            'Buy Price': buy_price,
            'Sold Price': sold_price_extended,
            'Profit/Loss': profit_loss_extended,
            'Hit Stop-Loss': hit_stop_loss_extended,
            'Hit Target-Profit': hit_target_profit_extended,
            'Exit Date': trade_window_extended.iloc[-1].name if trade_window_extended.iloc[-1].name < exit_date else exit_date
        })

    trade_results_extended_df = pd.DataFrame(trade_results_extended)

    total_trades_extended = len(trade_results_extended)
    winning_trades_extended = sum(1 for result in profit_losses_extended if result > 0)
    losing_trades_extended = total_trades_extended - winning_trades_extended
    win_rate_extended = winning_trades_extended / total_trades_extended * 100 if total_trades_extended else 0
    average_profit_loss_extended = sum(profit_losses_extended) / total_trades_extended if total_trades_extended else 0

    return trade_results_extended_df, {
        'Total Trades': total_trades_extended,
        'Winning Trades': winning_trades_extended,
        'Losing Trades': losing_trades_extended,
        'Win Rate (%)': win_rate_extended,
        'Average Profit/Loss (%)': average_profit_loss_extended * 100
    }



In [17]:
backtest(data, days_to_hold, stop_loss_threshold, target_profit_threshold)

(                 Entry Date   Buy Price  Sold Price  Profit/Loss  \
 0 2022-11-22 14:15:00+05:30  599.000000  601.299988     0.003840   
 1 2023-01-11 14:15:00+05:30  596.599976  536.939978    -0.100000   
 2 2023-01-30 15:15:00+05:30  541.299988  525.650024    -0.028912   
 3 2023-03-15 10:15:00+05:30  527.000000  578.299988     0.097343   
 4 2023-06-26 13:15:00+05:30  556.849976  572.799988     0.028643   
 5 2023-08-08 10:15:00+05:30  570.299988  600.299988     0.052604   
 6 2023-08-16 12:15:00+05:30  565.000000  598.299988     0.058938   
 7 2024-01-11 10:15:00+05:30  622.950012  760.349976     0.220563   
 
    Hit Stop-Loss  Hit Target-Profit                 Exit Date  
 0          False              False 2023-01-06 14:15:00+05:30  
 1           True              False 2023-02-24 15:15:00+05:30  
 2          False              False 2023-03-16 15:15:00+05:30  
 3          False              False 2023-04-28 15:15:00+05:30  
 4          False              False 2023-08-10 13:1